# Getting the info that the client needs for D3.js

Often it is easier to transform data in the backend to be used by D3. In general all data in D3 needs to be a list of dicts. In this example I'm using an existing `world` object to demonstrate the methods that do these transformations. 

In [1]:
#I'm mapping to the actual files in the repo so that I can also use this to troubleshoot
import os
import sys
import numpy as np
import pandas as pd
import pickle
import django
sys.path.append('../..')
#Loading my project settings from prodweb. This allows me to load and query models. 
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'prodweb.settings')
django.setup()


with (open("../pickles/billmanhworld.pkl", 'rb')) as pickle_file:
    world = pickle.load(pickle_file)

In [2]:
sys.path.append('..')
from lib.builders import world as w

`w.get_area_data(world)` gets the area relevant to where the character is. This includes surrounding areas.

In [3]:
world

In [4]:
print(world.Character.location)
pd.DataFrame(w.get_area_data(world)).T

[16, 33]


,aware,danger,elevation,feature,key,nation,nation number,rainfall,terrain,turn_last_visited,visited,x,y
area,1,1.005,42,Magerail,16:33,Ratspring,5,30,town,0,1,16,33
NArea,1,0.382,43,none,16:32,Ratspring,5,33,mountain,0,0,16,32
SArea,1,0.624,42,none,16:34,Ratspring,5,29,mountain,0,0,16,34
EArea,1,0.498,43,none,17:33,Ratspring,5,30,mountain,0,0,17,33
WArea,1,0.716,41,none,15:33,Ratspring,5,28,mountain,0,0,15,33


For other game principals, it's easy to just make regular `pandas` and `numpy` queries on those datasets. 

In [5]:
world.df_features.loc[world.Character.get_location_key(),'visited'] >= 1

True

Querying the terrain is easy as well. For example, here is a query of all of the world where the _terrain_ is _desert_ and the _danger_ is above .5. These queries are fast and cheap. 

In [6]:
world.df_features.loc[(world.df_features['terrain']=='desert') & 
                     (world.df_features['danger']>=.5)].head()

,y,rainfall,x,key,elevation,terrain,feature,nation number,nation,visited,aware,turn_last_visited,danger
key,,,,,,,,,,,,,
28:1,1,0.0,28,28:1,7.0,desert,NaN,3,Fenfort,0,0,0,0.612
29:1,1,0.0,29,29:1,8.0,desert,NaN,3,Fenfort,0,0,0,0.765
30:1,1,0.0,30,30:1,8.0,desert,NaN,3,Fenfort,0,0,0,0.846
34:1,1,0.0,34,34:1,8.0,desert,NaN,3,Fenfort,0,0,0,0.660
28:2,2,0.0,28,28:2,9.0,desert,NaN,3,Fenfort,0,0,0,0.825


You can also _mask_ the terrain where the character has no knowledge. 

In [7]:
masked_world = w.mask_unknown(world)
world.Character.title = "Noble"
masked_world.head()

,y,rainfall,x,key,elevation,terrain,feature,nation number,nation,visited,aware,turn_last_visited,danger
key,,,,,,,,,,,,,
0:0,0,2.0,0,0:0,1.0,ocean,NaN,2,Ratlight,0,1,0,-0.169
1:0,0,4.0,1,1:0,-2.0,ocean,NaN,2,Ratlight,0,1,0,-0.080
2:0,0,6.0,2,2:0,0.0,ocean,NaN,2,Ratlight,0,1,0,0.212
3:0,0,9.0,3,3:0,0.0,ocean,NaN,2,Ratlight,0,1,0,-0.102
4:0,0,10.0,4,4:0,0.0,ocean,NaN,2,Ratlight,0,1,0,0.310


## Analysis via list comprehension

Querying people is also easy, but you need to use _list comprehension_ to do it. This is the general structure. 

In [8]:
world.people


[Lakkar the Speaker of Stenchglen,
 Reppove the Speaker of Stormward,
 Truscoh the commoner,
 Depprith the commoner,
 Xigordur the Speaker of Oakvale,
 Nereshef the Speaker of Camloch,
 Iralordut the Speaker of Godrail,
 Anasassod the commoner,
 Drokkol the commoner,
 Botendis the Speaker of Fenglen,
 Minkim the Speaker of Foundersmoral,
 Prakrackorb the Speaker of Brinejaw,
 Brorandart the commoner,
 Ekinoddeb the commoner,
 Shestaker the Speaker of Ravenhenge,
 Nockish the Speaker of Warlair,
 Olarkol the commoner,
 Kiscottle the commoner,
 Adackolen the commoner,
 Eleklipove the commoner,
 Necklottle the Speaker of Fallhaven,
 Zhaldren the Speaker of Magerail,
 Afforb the Speaker of Servantshaven,
 Scakkich the commoner,
 Prankoe the commoner,
 Bestek the commoner,
 Dolordion the commoner,
 Femmark the Speaker of Clearhenge,
 Maklipar the Speaker of Lionhaven,
 Roldundle the commoner,
 Wrayellish the commoner,
 Mensof the commoner,
 Ovallir the commoner,
 Urbbenton the Speaker of Ma

Query the people in the town by setting conditions in the list. These people are the highest in temperment. 

In [9]:
[[(person.name,person.temperment) for person in t.get_population(world) if person.temperment>.5] for t in world.towns]

[[('Anasassod', 0.81), ('Scakkich', 0.99), ('Rarushett', 0.95)],
 [('Depprith', 0.9), ('Olarkol', 0.75)],
 [('Kiscottle', 0.84), ('Bestek', 0.87)],
 [('Nereshef', 0.66),
  ('Thezennumber', 0.53),
  ('Seblit', 0.71),
  ('Trafrec', 0.93)],
 [('Adackolen', 0.88), ('Dolordion', 0.54)],
 [],
 [('Minkim', 0.68)],
 [('Prakrackorb', 0.84), ('Roldundle', 0.95)],
 [('Shestaker', 0.61)],
 [('Mensof', 0.99), ('Akichot', 0.84)],
 [],
 [('Anasheh', 0.69)],
 [('Afforb', 0.72)],
 [],
 [('Maklipar', 0.92)],
 [('Urbbenton', 0.97), ('Joyangod', 0.73)],
 [('Femattaker', 0.8)],
 [('Elehottle', 0.72)],
 [('Hahilloh', 0.57)],
 [],
 []]

You can also get summary stats on towns. For example, the average temperment per town.

In [10]:
[(t.name,np.mean([person.temperment for person in t.get_population(world)])) for t in world.towns]

[('Stenchglen', 0.5085714285714286),
 ('Stormward', 0.46),
 ('Oakvale', 0.655),
 ('Camloch', 0.5860000000000001),
 ('Godrail', 0.4175),
 ('Fenglen', 0.23666666666666666),
 ('Foundersmoral', 0.68),
 ('Brinejaw', 0.5299999999999999),
 ('Ravenhenge', 0.33249999999999996),
 ('Warlair', 0.69),
 ('Fallhaven', 0.165),
 ('Magerail', 0.40499999999999997),
 ('Servantshaven', 0.43),
 ('Clearhenge', 0.09),
 ('Lionhaven', 0.39333333333333337),
 ('Mageward', 0.7200000000000001),
 ('Kingscraft', 0.455),
 ('Fallwish', 0.53),
 ('Magecrest', 0.57),
 ('Kingsjaw', 0.01),
 ('Ravenvale', 0.34)]

For some purposes you may want to get a collection of people and edit them in bulk. This is also possible using list comprehension. 

In [11]:
w.get_people_where_char_has_visited(world)

[{'town': capitol of Magerail. Location: [16:33]. Founded 1111 </br>In the nation of Ratspring,
  'people': [Zhaldren the Speaker of Magerail,
   Anasheh the ruler of Ratspring]}]

In [12]:
r = w.get_people_where_char_has_visited(world)[0]
r

{'town': capitol of Magerail. Location: [16:33]. Founded 1111 </br>In the nation of Ratspring,
 'people': [Zhaldren the Speaker of Magerail, Anasheh the ruler of Ratspring]}

In [13]:
r['town'].nation

'Ratspring'

In [14]:
w.get_relationships_node_map(world)

[{'name': 'Magerail',
  'title': 'capitol of Magerail. Location',
  'nation': 'Ratspring',
  'population': 2,
  'type': 'capitol',
  'location': '16:33',
  'founded year': 1111,
  'children': [{'name': 'Zhaldren',
    'role': 'Speaker of Magerail',
    'temperment': 0.12,
    'loyalty': 0.5,
    'type': 'person'},
   {'name': 'Anasheh',
    'role': 'ruler of Ratspring',
    'temperment': 0.69,
    'loyalty': 1,
    'type': 'person'}]}]

## Getting map data (mapData): 

You can use this to get lists of people who the character knows. 

In [15]:
[(T.nation,T.name) for T in world.towns]

[('Lilyloch', 'Stenchglen'),
 ('Earthkeep', 'Stormward'),
 ('Lilyloch', 'Oakvale'),
 ('Landshost', 'Camloch'),
 ('Fenfort', 'Godrail'),
 ('Malhail', 'Fenglen'),
 ('Earthkeep', 'Foundersmoral'),
 ('Brinetown', 'Brinejaw'),
 ('Landshost', 'Ravenhenge'),
 ('Fenfort', 'Warlair'),
 ('Landshost', 'Fallhaven'),
 ('Ratspring', 'Magerail'),
 ('Brinetown', 'Servantshaven'),
 ('Lilyloch', 'Clearhenge'),
 ('Lilyloch', 'Lionhaven'),
 ('Ratlight', 'Mageward'),
 ('Lilyloch', 'Kingscraft'),
 ('Earthkeep', 'Fallwish'),
 ('Ratlight', 'Magecrest'),
 ('Malhail', 'Kingsjaw'),
 ('Brinetown', 'Ravenvale')]

In [16]:
masked_world.loc[world.Character.get_location_key()]

y                           33
rainfall                    30
x                           16
key                      16:33
elevation                   42
terrain                   town
feature               Magerail
nation number                5
nation               Ratspring
visited                      1
aware                        1
turn_last_visited            0
danger                   1.005
Name: 16:33, dtype: object

You can query nations or towns. `world.nations` is a list of _nations_ and `world.towns` is a list of _towns_

In [17]:
where_the_char_has_been = world.df_features.loc[(world.df_features['visited']==1)&
                                                (world.df_features['terrain']=='town')]
where_the_char_has_been

,y,rainfall,x,key,elevation,terrain,feature,nation number,nation,visited,aware,turn_last_visited,danger
key,,,,,,,,,,,,,
16:33,33,30.0,16,16:33,42.0,town,Magerail,5,Ratspring,1,1,0,1.005


## Getting the diplomacy and relationships data for the journal page

In [18]:
rel = w.get_people_where_char_has_visited(world)
rel

[{'town': capitol of Magerail. Location: [16:33]. Founded 1111 </br>In the nation of Ratspring,
  'people': [Zhaldren the Speaker of Magerail,
   Anasheh the ruler of Ratspring]}]

In [19]:
w.get_relationships_node_map(world)

[{'name': 'Magerail',
  'title': 'capitol of Magerail. Location',
  'nation': 'Ratspring',
  'population': 2,
  'type': 'capitol',
  'location': '16:33',
  'founded year': 1111,
  'children': [{'name': 'Zhaldren',
    'role': 'Speaker of Magerail',
    'temperment': 0.12,
    'loyalty': 0.5,
    'type': 'person'},
   {'name': 'Anasheh',
    'role': 'ruler of Ratspring',
    'temperment': 0.69,
    'loyalty': 1,
    'type': 'person'}]}]

In [20]:
hierarchy = [{'nation':i.name,
              'children':i.diplomacy} for i in world.nations]



In [21]:
relationships = w.get_relationships_node_map(world)
relationships

[{'name': 'Magerail',
  'title': 'capitol of Magerail. Location',
  'nation': 'Ratspring',
  'population': 2,
  'type': 'capitol',
  'location': '16:33',
  'founded year': 1111,
  'children': [{'name': 'Zhaldren',
    'role': 'Speaker of Magerail',
    'temperment': 0.12,
    'loyalty': 0.5,
    'type': 'person'},
   {'name': 'Anasheh',
    'role': 'ruler of Ratspring',
    'temperment': 0.69,
    'loyalty': 1,
    'type': 'person'}]}]

In [22]:
nations = [s['nation'] for s in relationships]
nations

['Ratspring']

In [23]:
[i for i in relationships]

[{'name': 'Magerail',
  'title': 'capitol of Magerail. Location',
  'nation': 'Ratspring',
  'population': 2,
  'type': 'capitol',
  'location': '16:33',
  'founded year': 1111,
  'children': [{'name': 'Zhaldren',
    'role': 'Speaker of Magerail',
    'temperment': 0.12,
    'loyalty': 0.5,
    'type': 'person'},
   {'name': 'Anasheh',
    'role': 'ruler of Ratspring',
    'temperment': 0.69,
    'loyalty': 1,
    'type': 'person'}]}]

In [24]:
[{'nation':i,'towns':[t for t in relationships if t['nation']==i]} for i in nations]

[{'nation': 'Ratspring',
  'towns': [{'name': 'Magerail',
    'title': 'capitol of Magerail. Location',
    'nation': 'Ratspring',
    'population': 2,
    'type': 'capitol',
    'location': '16:33',
    'founded year': 1111,
    'children': [{'name': 'Zhaldren',
      'role': 'Speaker of Magerail',
      'temperment': 0.12,
      'loyalty': 0.5,
      'type': 'person'},
     {'name': 'Anasheh',
      'role': 'ruler of Ratspring',
      'temperment': 0.69,
      'loyalty': 1,
      'type': 'person'}]}]}]

In [25]:
[i for i in nations]

['Ratspring']

In [36]:
def get_all_nodes(world):
    people = [{'id':str(t),'type':'person'} for t in world.people]
    nations = [{'id':str(t),'type':'nation'} for t in world.nations]
    towns = [{'id':f"{t.type} of {t.name}",'type':'town'} for t in world.towns]
    return towns + nations + people

def get_all_links(world):
    all_links = []
    town_links = [[all_links.append({'source':str(n),'target':f"{t.type} of {t.name}"}) for t in n.get_all_towns(world)] for n in world.nations]
    people_links = [[all_links.append({'source':f"{n.type} of {n.name}",'target':str(t)}) for t in n.get_population(world)] for n in world.towns]
    return all_links


def get_nodes_and_links(world):
    return {'nodes':get_all_nodes(world),
           'links':get_all_links(world)}

get_nodes_and_links(world)

{'nodes': [{'id': 'capitol of Stenchglen', 'type': 'town'},
  {'id': 'capitol of Stormward', 'type': 'town'},
  {'id': 'town of Oakvale', 'type': 'town'},
  {'id': 'capitol of Camloch', 'type': 'town'},
  {'id': 'capitol of Godrail', 'type': 'town'},
  {'id': 'capitol of Fenglen', 'type': 'town'},
  {'id': 'town of Foundersmoral', 'type': 'town'},
  {'id': 'capitol of Brinejaw', 'type': 'town'},
  {'id': 'town of Ravenhenge', 'type': 'town'},
  {'id': 'town of Warlair', 'type': 'town'},
  {'id': 'town of Fallhaven', 'type': 'town'},
  {'id': 'capitol of Magerail', 'type': 'town'},
  {'id': 'town of Servantshaven', 'type': 'town'},
  {'id': 'town of Clearhenge', 'type': 'town'},
  {'id': 'town of Lionhaven', 'type': 'town'},
  {'id': 'capitol of Mageward', 'type': 'town'},
  {'id': 'town of Kingscraft', 'type': 'town'},
  {'id': 'town of Fallwish', 'type': 'town'},
  {'id': 'town of Magecrest', 'type': 'town'},
  {'id': 'town of Kingsjaw', 'type': 'town'},
  {'id': 'town of Ravenvale', 